# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [7]:
import requests
from bs4 import BeautifulSoup

html = requests.get("https://books.toscrape.com/catalogue/page-1.html", timeout=10).content
soup = BeautifulSoup(html, "html.parser")
pods = soup.select("article.product_pod")
print("books on page 1:", len(pods))

books on page 1: 20


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

STAR_MAP = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

def get_soup(url):
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    return BeautifulSoup(r.content, "html.parser")

def scrape_books(min_rating=4, max_price=20, max_pages=3):
    base = "https://books.toscrape.com/"
    page_url = base + "catalogue/page-1.html"
    rows = []
    page = 1

    while True:
        print(f"Procesing page {page}: {page_url}") 
        try:
            soup = get_soup(page_url)
        except Exception as e:
            print("Error loading page:", e)
            break

        pods = soup.select("article.product_pod")
        if not pods:
            print("No books in this page.")
            break

        for pod in pods:
            title = pod.h3.a["title"]
            price_text = pod.select_one("p.price_color").get_text(strip=True)
            price = float(price_text.replace("£", ""))
            rating_class = pod.select_one("p.star-rating")["class"]
            rating = STAR_MAP.get(next(c for c in rating_class if c != "star-rating"), 0)

            detail_rel = pod.h3.a["href"]
            if detail_rel.startswith("../"):
                detail_rel = detail_rel.replace("../", "")
            detail_url = base + "catalogue/" + detail_rel

            try:
                d = get_soup(detail_url)
            except Exception as e:
                print("Error detail, omits:", e)
                continue

            upc = d.select_one("table.table.table-striped tr:nth-of-type(1) td").get_text(strip=True)
            availability = d.select_one("p.instock.availability").get_text(strip=True)
            desc_header = d.select_one("#product_description")
            description = desc_header.find_next("p").get_text(strip=True) if desc_header else ""
            crumbs = d.select("ul.breadcrumb li a")
            genre = crumbs[2].get_text(strip=True) if len(crumbs) > 2 else "Unknown"

            if rating >= min_rating and price <= max_price:
                rows.append({
                    "UPC": upc,
                    "Title": title,
                    "Price (£)": price,
                    "Rating": rating,
                    "Genre": genre,
                    "Availability": availability,
                    "Description": description
                })

        next_li = soup.select_one("li.next a")
        if not next_li:
            break

        href = next_li["href"]              
        page_url = base + "catalogue/" + href
        page += 1
        if page > max_pages:                
            print(f"Limit found pages ({max_pages}).")
            break

    return pd.DataFrame(rows)

In [ ]:
df = scrape_books(min_rating=4, max_price=20, max_pages=2)
print(df.head())
print(len(df), "Found Books")